In [1]:
# -*- coding: utf-8 -*-

import os
os.chdir('/home/zl/develop/projects/hbc')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import time
import datetime
%matplotlib inline

In [2]:
### define some query functions
def getTimespanWithHolidays(yday, yday_holi_list):
    if len(yday_holi_list) <= 0:
        return 100
    timespan2holi_min = 1000
    for i in range(len(yday_holi_list)):
        timespan2holi_tmp = np.abs(yday_holi_list[i] - yday)
        if timespan2holi_tmp < timespan2holi_min:
            timespan2holi_min = timespan2holi_tmp
    return timespan2holi_min

def getTimeFromTimestamp(timestamp):
    time_local = time.localtime(timestamp)
    tm_year = time_local.tm_year
    tm_mon = time_local.tm_mon
    tm_day = time_local.tm_mday
    tm_hour = time_local.tm_hour
    tm_min = time_local.tm_min
    tm_sec = time_local.tm_sec
    tm_wday = time_local.tm_wday
    tm_yday = time_local.tm_yday
    return tm_year, tm_mon, tm_day, tm_hour, tm_min, tm_sec, tm_wday, tm_yday

def getTagsFromOrderHistoryByUserid(df_ordhistory, userid):
    df_ordhistory_of_userid = df_ordhistory[df_ordhistory['userid'] == userid]
    sum_all = len(df_ordhistory_of_userid)    # 购买过旅游服务的次数
    sum0 = np.sum(df_ordhistory_of_userid['orderType'] == 0)    # 购买过普通旅游服务的次数
    sum1 = np.sum(df_ordhistory_of_userid['orderType'] == 1)    # 购买过精品旅游服务的次数
    ratio1_0 = (sum1+1.0) / (sum0+1.0)                  # 精品/普通(比例)
    time_last_1 = np.nan#0#np.min(df_ordhistory['orderTime'])      # 最后一次订单的时间
    time_last_1_year = np.nan
    time_last_1_month = np.nan
    time_last_1_hour = np.nan
    time_last_1_yday = np.nan
    time_last_2 = np.nan
    time_last_2_year = np.nan
    time_last_2_month = np.nan
    time_last_2_hour = np.nan
    time_last_2_yday = np.nan
    time_last_3 = np.nan
    time_last_3_year = np.nan
    time_last_3_month = np.nan
    time_last_3_hour = np.nan
    time_last_3_yday = np.nan
    sum_cont1 = 0
    sum_cont2 = 0
    sum_cont3 = 0
    sum_cont4 = 0
    sum_cont5 = 0
    #sum_countries = pd.DataFrame()
    #sum_cities = pd.DataFrame()
#    ratio_countries = pd.DataFrame()
#    ratio_cities = pd.DataFrame()
    ordtime_mean = np.nan
    ordtime_min = np.nan
    ordtime_max = np.nan
    ordtime_std = np.nan
    time_ordtype1_last_1 = np.nan
    time_ordtype1_last_2 = np.nan
    ord_last_1_type = np.nan
    ord_last_2_type = np.nan
    ord_last_3_type = np.nan
    continent_last_1 = 'continent_nan'
    country_last_1 = 'country_nan'
    city_last_1 = 'city_nan'
    #for k in range(len(countryset)):
    #    sum_countries[countryset[k]] = [0]
    #for k in range(len(cityset)):
    #    sum_cities[cityset[k]] = [0]
    if len(df_ordhistory_of_userid) > 0:
        df_ordhistory_of_userid_sorted = df_ordhistory_of_userid.sort_values(by=['orderTime'])
        ord_last_1_type = df_ordhistory_of_userid_sorted['orderType'].iloc[-1]
        time_last_1 = np.max(df_ordhistory_of_userid['orderTime'])
        time_local = time.localtime(time_last_1)
        time_last_1_year = time_local.tm_year
        time_last_1_month = time_local.tm_mon
        time_last_1_hour = time_local.tm_hour
        time_last_1_yday = time_local.tm_yday
        df_ordhistory_of_userid_sorted_ordtype1 = df_ordhistory_of_userid.sort_values(by=['orderTime'])[df_ordhistory_of_userid.sort_values(by=['orderTime'])['orderType'] == 1]
        continent_last_1 = df_ordhistory_of_userid_sorted['continent'].iat[-1]
        country_last_1 = df_ordhistory_of_userid_sorted['country'].iat[-1]
        city_last_1 = df_ordhistory_of_userid_sorted['city'].iat[-1]
        #if len(df_ordhistory_of_userid) > 0:
        #    sum_cont1 = np.sum(df_ordhistory_of_userid['continent'] == '亚洲')
        #    sum_cont2 = np.sum(df_ordhistory_of_userid['continent'] == '欧洲')
        #    sum_cont3 = np.sum(df_ordhistory_of_userid['continent'] == '北美洲')
        #    sum_cont4 = np.sum(df_ordhistory_of_userid['continent'] == '大洋洲')
        #    sum_cont5 = np.sum(df_ordhistory_of_userid['continent'] == '非洲')
        #    for k in range(len(countryset)):
        #        sum_countries[countryset[k]] = [np.sum(df_ordhistory_of_userid['country'] == countryset[k])]
        #    for k in range(len(cityset)):
        #        sum_cities[cityset[k]] = [np.sum(df_ordhistory_of_userid['city'] == cityset[k])]
#            for k in range(len(countryset)):
#                ratio_countries[countryset[k]] = np.sum((df_ordhistory_of_userid['country'] == countryset[k]) & (df_ordhistory_of_userid['orderType'] == 1)) / (np.sum(df_ordhistory_of_userid['country'] == countryset[k])+0.01)
#            for k in range(len(cityset)):
#                ratio_cities[cityset[k]] = np.sum((df_ordhistory_of_userid['city'] == cityset[k]) & (df_ordhistory_of_userid['orderType'] == 1)) / (np.sum(df_ordhistory_of_userid['city'] == cityset[k])+0.01)
        if len(df_ordhistory_of_userid_sorted_ordtype1) > 0:
            time_ordtype1_last_1 = df_ordhistory_of_userid_sorted_ordtype1['orderTime'].iat[-1]
        if len(df_ordhistory_of_userid_sorted_ordtype1) > 1:
            time_ordtype1_last_2 = df_ordhistory_of_userid_sorted_ordtype1['orderTime'].iat[-2]
    if len(df_ordhistory_of_userid) > 1:
        time_last_2 = df_ordhistory_of_userid.sort_values(by=['orderTime'])['orderTime'].iat[-2]
        time_local = time.localtime(time_last_2)
        time_last_2_year = time_local.tm_year
        time_last_2_month = time_local.tm_mon
        time_last_2_hour = time_local.tm_hour
        time_last_2_yday = time_local.tm_yday
        df_ordhistory_of_userid_sorted = df_ordhistory_of_userid.sort_values(by=['orderTime'])
        ord_last_2_type = df_ordhistory_of_userid_sorted['orderType'].iloc[-2]
    if len(df_ordhistory_of_userid) > 2:
        time_last_3 = df_ordhistory_of_userid.sort_values(by=['orderTime'])['orderTime'].iat[-3]
        time_local = time.localtime(time_last_3)
        time_last_3_year = time_local.tm_year
        time_last_3_month = time_local.tm_mon
        time_last_3_hour = time_local.tm_hour
        time_last_3_yday = time_local.tm_yday
        timelist = np.array(df_ordhistory_of_userid.sort_values(by=['orderTime'])['orderTime'])
        timespanlist = []
        for i in range(1, len(timelist)):
            timespanlist.append(timelist[i] - timelist[i-1])
        ordtime_mean = np.mean(timespanlist)
        ordtime_min = np.min(timespanlist)
        ordtime_max = np.max(timespanlist)
        ordtime_std = np.std(timespanlist)
        df_ordhistory_of_userid_sorted = df_ordhistory_of_userid.sort_values(by=['orderTime'])
        ord_last_3_type = df_ordhistory_of_userid_sorted['orderType'].iloc[-3]
    return sum_all, sum0, sum1, ratio1_0, time_last_1, time_last_1_year, time_last_1_month, time_last_1_hour, time_last_1_yday, time_last_2, time_last_2_year, time_last_2_month, time_last_2_hour, time_last_2_yday, time_last_3, time_last_3_year, time_last_3_month, time_last_3_hour, time_last_3_yday, ordtime_mean, ordtime_min, ordtime_max, ordtime_std, time_ordtype1_last_1, time_ordtype1_last_2, ord_last_1_type, ord_last_2_type, ord_last_3_type, continent_last_1, country_last_1, city_last_1

def getActionTimeSpanMean(df_action_of_userid, actiontypeA, actiontypeB):
    timespan_list = []
    i = 0
    while i < (len(df_action_of_userid)-1):
        if df_action_of_userid['actionType'].iat[i] == actiontypeA:
            timeA = df_action_of_userid['actionTime'].iat[i]
            for j in range(i+1, len(df_action_of_userid)):
                if df_action_of_userid['actionType'].iat[j] == actiontypeA:
                    timeA = df_action_of_userid['actionTime'].iat[j]
                if df_action_of_userid['actionType'].iat[j] == actiontypeB:
                    timeB = df_action_of_userid['actionTime'].iat[j]
                    timespan_list.append(timeB-timeA)
                    i = j
                    break
        i+=1
    if len(timespan_list) > 0:
        return np.mean(timespan_list)
    else:
        return np.nan

def getActionTimeSpanCount(df_action_of_userid, actiontypeA, actiontypeB, timethred=100):
    timespan_list = []
    i = 0
    while i < (len(df_action_of_userid)-1):
        if df_action_of_userid['actionType'].iat[i] == actiontypeA:
            timeA = df_action_of_userid['actionTime'].iat[i]
            for j in range(i+1, len(df_action_of_userid)):
                if df_action_of_userid['actionType'].iat[j] == actiontypeA:
                    timeA = df_action_of_userid['actionTime'].iat[j]
                if df_action_of_userid['actionType'].iat[j] == actiontypeB:
                    timeB = df_action_of_userid['actionTime'].iat[j]
                    timespan_list.append(timeB-timeA)
                    i = j
                    break
        i+=1
    return np.sum(np.array(timespan_list) <= timethred)

def get2ActionTimeSpanLast(df_action_of_userid, actiontypeA, actiontypeB, timespan_max=10000):
    timespan_list = []
    i = 0
    while i < (len(df_action_of_userid)-1):
        if df_action_of_userid['actionType'].iat[i] == actiontypeA:
            timeA = df_action_of_userid['actionTime'].iat[i]
            for j in range(i+1, len(df_action_of_userid)):
                if df_action_of_userid['actionType'].iat[j] == actiontypeA:
                    timeA = df_action_of_userid['actionTime'].iat[j]
                    continue
                if df_action_of_userid['actionType'].iat[j] == actiontypeB:
                    timeB = df_action_of_userid['actionTime'].iat[j]
                    timespan_list.append(timeB-timeA)
                    i = j
                    break
        i+=1
    if len(timespan_list) > 0:
        timespan = timespan_list[-1]
        if timespan > timespan_max:
            timespan = timespan_max
        return timespan
    else:
        return np.nan

def getAction2LastTimeSpanList(df_action_of_userid, actiontype):
    timespan_list = []
    i = 0
    actionlength = len(df_action_of_userid)
    if actionlength <= 1:
        return []
    actionIndex = 0
    while i < actionlength:
        if df_action_of_userid['actionType'].iat[i] == actiontype:
            actionIndex = i
        i += 1
    if actionIndex == 0:
        return []
    if actionIndex >= (actionlength-1):
        timespan_list = [0.0]
        return timespan_list
    if actionIndex < (actionlength-1):
        for j in range(actionIndex, actionlength-1):
            timeA = df_action_of_userid['actionTime'].iat[j]
            timeB = df_action_of_userid['actionTime'].iat[j+1]
            timespan_list.append(timeB-timeA)
    return timespan_list

def calc_seqentialratio(df_action_of_userid):
    i = 0
    pos_5 = -1
    result = 0
    df_len = len(df_action_of_userid)
    for i in range(0, df_len):
        if df_action_of_userid['actionType'].iat[i] == 5:
            pos_5 = i
    if pos_5 != -1:
        result += 1
        if pos_5+1 < df_len:
            if df_action_of_userid['actionType'].iat[pos_5+1] == 6:
                result += 1
                if pos_5+2 < df_len:
                    if df_action_of_userid['actionType'].iat[pos_5+2] == 7:
                        result += 1
                        if pos_5+3 < df_len:
                            if df_action_of_userid['actionType'].iat[pos_5+3] == 8:
                                result += 1
    return result

def getTagsFromActionByUserid(df_action, userid):
    df_action_of_userid = df_action[df_action['userid'] == userid]
    sum_action = len(df_action_of_userid)  # 一个用户action的总次数
    actiontime_last_1_year = np.nan
    actiontime_last_1_month = np.nan
    action_last_1 = np.nan   # 倒数第1次actionType
    action_last_1 = np.nan   # 倒数第1次actionType
    action_last_2 = np.nan   # 倒数第2次actionType
    action_last_3 = np.nan   # 倒数第3次actionType
    action_last_4 = np.nan   # 倒数第4次actionType
    action_last_5 = np.nan   # 倒数第5次actionType
    action_last_6 = np.nan   # 倒数第6次actionType
    action_last_7 = np.nan   # 倒数第7次actionType
    action_last_8 = np.nan   # 倒数第8次actionType
    action_last_9 = np.nan   # 倒数第9次actionType
    action_last_10 = np.nan  # 倒数第10次actionType
    action_last_11 = np.nan  # 倒数第11次actionType
    action_last_12 = np.nan  # 倒数第12次actionType
    action_last_13 = np.nan  # 倒数第13次actionType
    action_last_14 = np.nan  # 倒数第14次actionType
    action_last_15 = np.nan  # 倒数第15次actionType
    action_last_16 = np.nan  # 倒数第16次actionType
    action_last_17 = np.nan  # 倒数第17次actionType
    action_last_18 = np.nan  # 倒数第18次actionType
    action_last_19 = np.nan  # 倒数第19次actionType
    action_last_20 = np.nan  # 倒数第20次actionType
    #actiontime_mean = np.mean(df_action['actionTime'])
    actiontime_last_1 = np.nan   # 倒数第1次actionTime
    actiontime_last_2 = np.nan   # 倒数第2次actionTime
    actiontime_last_3 = np.nan   # 倒数第3次actionTime
    actiontime_last_4 = np.nan   # 倒数第4次actionTime
    actiontime_last_5 = np.nan   # 倒数第5次actionTime
    actiontime_last_6 = np.nan   # 倒数第6次actionTime
    actiontime_last_7 = np.nan   # 倒数第7次actionTime
    actiontime_last_8 = np.nan   # 倒数第8次actionTime
    actiontime_last_9 = np.nan   # 倒数第9次actionTime
    actiontime_last_10 = np.nan  # 倒数第10次actionTime
    actiontime_last_11 = np.nan  # 倒数第11次actionTime
    actiontime_last_12 = np.nan  # 倒数第12次actionTime
    actiontime_last_13 = np.nan  # 倒数第13次actionTime
    actiontime_last_14 = np.nan  # 倒数第14次actionTime
    actiontime_last_15 = np.nan  # 倒数第15次actionTime
    actiontime_last_16 = np.nan  # 倒数第16次actionTime
    actiontime_last_17 = np.nan  # 倒数第17次actionTime
    actiontime_last_18 = np.nan  # 倒数第18次actionTime
    actiontime_last_19 = np.nan  # 倒数第19次actionTime
    actiontime_last_20 = np.nan  # 倒数第20次actionTime
    actiontypeprop_1 = 0  # actionType1占比
    actiontypeprop_2 = 0  # actionType2占比
    actiontypeprop_3 = 0  # actionType3占比
    actiontypeprop_4 = 0  # actionType4占比
    actiontypeprop_5 = 0  # actionType5占比
    actiontypeprop_6 = 0  # actionType6占比
    actiontypeprop_7 = 0  # actionType7占比
    actiontypeprop_8 = 0  # actionType8占比
    actiontypeprop_9 = 0  # actionType9占比
    timespanthred = 100
    actiontimespancount_1_5 = 0  # actionType1-5的时间差小于timespanthred的数量
    actiontimespancount_5_6 = 0  # actionType5-6的时间差小于timespanthred的数量
    actiontimespancount_6_7 = 0  # actionType6-7的时间差小于timespanthred的数量
    actiontimespancount_7_8 = 0  # actionType7-8的时间差小于timespanthred的数量
    actiontimespancount_8_9 = 0  # actionType8-9的时间差小于timespanthred的数量
    actionratio_24_59 = np.nan      # actionType2-4与5-9之间的比值
    actiontype_lasttime_1 = np.nan    # actionType1最后一次出现的时间
    actiontype_lasttime_5 = np.nan    # actionType5最后一次出现的时间
    actiontype_lasttime_6 = np.nan    # actionType6最后一次出现的时间
    actiontype_lasttime_7 = np.nan    # actionType7最后一次出现的时间
    actiontype_lasttime_8 = np.nan    # actionType8最后一次出现的时间
    actiontype_lasttime_9 = np.nan    # actionType9最后一次出现的时间
    actiontype_lasttime_24 = np.nan   # actionType2-4最后一次出现的时间
    actiontimespanlast_1_5 = np.nan   # 最后一次actionType1与5之间的间隔
    actiontimespanlast_5_6 = np.nan   # 最后一次actionType5与6之间的间隔
    actiontimespanlast_6_7 = np.nan   # 最后一次actionType6与7之间的间隔
    actiontimespanlast_7_8 = np.nan   # 最后一次actionType7与8之间的间隔
    actiontimespanlast_5_7 = np.nan   # 最后一次actionType5与7之间的间隔
    actiontimespanlast_5_8 = np.nan   # 最后一次actionType5与8之间的间隔
    action59seqentialratio = 0      # actionType5-9的连续程度
    actiontypeproplast10_1 = 0  # 最后10个action中，actionType1占比
    actiontypeproplast10_2 = 0  # 最后10个action中，actionType2占比
    actiontypeproplast10_3 = 0  # 最后10个action中，actionType3占比
    actiontypeproplast10_4 = 0  # 最后10个action中，actionType4占比
    actiontypeproplast10_5 = 0  # 最后10个action中，actionType5占比
    actiontypeproplast10_6 = 0  # 最后10个action中，actionType6占比
    actiontypeproplast10_7 = 0  # 最后10个action中，actionType7占比
    actiontypeproplast10_8 = 0  # 最后10个action中，actionType8占比
    actiontypeproplast10_9 = 0  # 最后10个action中，actionType9占比
    actiontime_1 = np.nan            # 第一个actionTime（用户第一次使用app的时间）
    actiontimespancount_last10_1_5 = 0  # 最后10个action中actionType1-5的时间差小于timespanthred的数量
    actiontimespancount_last10_5_6 = 0  # 最后10个action中actionType5-6的时间差小于timespanthred的数量
    actiontimespancount_last10_6_7 = 0  # 最后10个action中actionType6-7的时间差小于timespanthred的数量
    actiontimespancount_last10_7_8 = 0  # 最后10个action中actionType7-8的时间差小于timespanthred的数量
    actiontimespancount_last10_8_9 = 0  # 最后10个action中actionType8-9的时间差小于timespanthred的数量
    actiontimespan_mean_1_5 = np.nan        # 所有的action中1-5的平均时间间隔
    actiontimespan_mean_5_9 = np.nan        # 所有的action中5-9的平均时间间隔
    actiontimespan_mean_1_9 = np.nan        # 所有的action中1-9的平均时间间隔
    timespan2last_action1_len = np.nan     # 每个actiontype的最后一个到最后一个action之间的时间差列表，获取其长度、平均值、标准差
    timespan2last_action5_len = np.nan
    timespan2last_action6_len = np.nan
    timespan2last_action7_len = np.nan
    timespan2last_action8_len = np.nan
    timespan2last_action9_len = np.nan
    timespan2last_action1_mean = np.nan
    timespan2last_action5_mean = np.nan
    timespan2last_action6_mean = np.nan
    timespan2last_action7_mean = np.nan
    timespan2last_action8_mean = np.nan
    timespan2last_action9_mean = np.nan
    timespan2last_action1_std = np.nan
    timespan2last_action5_std = np.nan
    timespan2last_action6_std = np.nan
    timespan2last_action7_std = np.nan
    timespan2last_action8_std = np.nan
    timespan2last_action9_std = np.nan
    if sum_action >= 1:
        actiontime_1 = df_action_of_userid['actionTime'].iat[0]
        action_last_1 = df_action_of_userid['actionType'].iat[-1]
        actiontime_last_1 = df_action_of_userid['actionTime'].iat[-1]
        time_local = time.localtime(actiontime_last_1)
        actiontime_last_1_year = time_local.tm_year
        actiontime_last_1_month = time_local.tm_mon
        if len(df_action_of_userid[df_action_of_userid['actionType'] == 1]) > 0:
            actiontype_lasttime_1 = df_action_of_userid[df_action_of_userid['actionType'] == 1].iloc[-1]['actionTime']
        if len(df_action_of_userid[df_action_of_userid['actionType'] == 5]) > 0:
            actiontype_lasttime_5 = df_action_of_userid[df_action_of_userid['actionType'] == 5].iloc[-1]['actionTime']
        if len(df_action_of_userid[df_action_of_userid['actionType'] == 6]) > 0:
            actiontype_lasttime_6 = df_action_of_userid[df_action_of_userid['actionType'] == 6].iloc[-1]['actionTime']
        if len(df_action_of_userid[df_action_of_userid['actionType'] == 7]) > 0:
            actiontype_lasttime_7 = df_action_of_userid[df_action_of_userid['actionType'] == 7].iloc[-1]['actionTime']
        if len(df_action_of_userid[df_action_of_userid['actionType'] == 8]) > 0:
            actiontype_lasttime_8 = df_action_of_userid[df_action_of_userid['actionType'] == 8].iloc[-1]['actionTime']
        if len(df_action_of_userid[df_action_of_userid['actionType'] == 9]) > 0:
            actiontype_lasttime_9 = df_action_of_userid[df_action_of_userid['actionType'] == 9].iloc[-1]['actionTime']
        if len(df_action_of_userid[(df_action_of_userid['actionType'] >= 2) & (df_action_of_userid['actionType'] <= 4)]) > 0:    
            actiontype_lasttime_24 = df_action_of_userid[(df_action_of_userid['actionType'] >= 2) & (df_action_of_userid['actionType'] <= 4)].iloc[-1]['actionTime']
    if sum_action >= 2:
        action_last_2 = df_action_of_userid['actionType'].iat[-2]
        actiontime_last_2 = df_action_of_userid['actionTime'].iat[-2]
    if sum_action >= 3:
        action_last_3 = df_action_of_userid['actionType'].iat[-3]
        actiontime_last_3 = df_action_of_userid['actionTime'].iat[-3]
        actiontimespanlast_1_5 = get2ActionTimeSpanLast(df_action_of_userid, 1, 5)
        actiontimespanlast_5_6 = get2ActionTimeSpanLast(df_action_of_userid, 5, 6)
        actiontimespanlast_6_7 = get2ActionTimeSpanLast(df_action_of_userid, 6, 7)
        actiontimespanlast_7_8 = get2ActionTimeSpanLast(df_action_of_userid, 7, 8)
        actiontimespanlast_5_7 = get2ActionTimeSpanLast(df_action_of_userid, 5, 7)
        actiontimespanlast_5_8 = get2ActionTimeSpanLast(df_action_of_userid, 5, 8)
        action59seqentialratio = calc_seqentialratio(df_action_of_userid)
        # 每个actiontype的最后一个到最后一个action之间的时间差列表
        timespan_actoin1tolastlist = getAction2LastTimeSpanList(df_action_of_userid, 1)
        timespan_actoin5tolastlist = getAction2LastTimeSpanList(df_action_of_userid, 5)
        timespan_actoin6tolastlist = getAction2LastTimeSpanList(df_action_of_userid, 6)
        timespan_actoin7tolastlist = getAction2LastTimeSpanList(df_action_of_userid, 7)
        timespan_actoin8tolastlist = getAction2LastTimeSpanList(df_action_of_userid, 8)
        timespan_actoin9tolastlist = getAction2LastTimeSpanList(df_action_of_userid, 9)
        timespan2last_action1_len = len(timespan_actoin1tolastlist)
        timespan2last_action5_len = len(timespan_actoin5tolastlist)
        timespan2last_action6_len = len(timespan_actoin6tolastlist)
        timespan2last_action7_len = len(timespan_actoin7tolastlist)
        timespan2last_action8_len = len(timespan_actoin8tolastlist)
        timespan2last_action9_len = len(timespan_actoin9tolastlist)
        if timespan2last_action1_len > 0:
            timespan2last_action1_mean = np.mean(timespan_actoin1tolastlist)
            timespan2last_action1_std = np.std(timespan_actoin1tolastlist)
        if timespan2last_action5_len > 0:
            timespan2last_action5_mean = np.mean(timespan_actoin5tolastlist)
            timespan2last_action5_std = np.std(timespan_actoin5tolastlist)
        if timespan2last_action6_len > 0:
            timespan2last_action6_mean = np.mean(timespan_actoin6tolastlist)
            timespan2last_action6_std = np.std(timespan_actoin6tolastlist)
        if timespan2last_action7_len > 0:
            timespan2last_action7_mean = np.mean(timespan_actoin7tolastlist)
            timespan2last_action7_std = np.std(timespan_actoin7tolastlist)
        if timespan2last_action8_len > 0:
            timespan2last_action8_mean = np.mean(timespan_actoin8tolastlist)
            timespan2last_action8_std = np.std(timespan_actoin8tolastlist)
        if timespan2last_action9_len > 0:
            timespan2last_action9_mean = np.mean(timespan_actoin9tolastlist)
            timespan2last_action9_std = np.std(timespan_actoin9tolastlist) 
    if sum_action >= 4:
        action_last_4 = df_action_of_userid['actionType'].iat[-4]
        actiontime_last_4 = df_action_of_userid['actionTime'].iat[-4]
    if sum_action >= 5:
        action_last_5 = df_action_of_userid['actionType'].iat[-5]
        actiontime_last_5 = df_action_of_userid['actionTime'].iat[-5]
    if sum_action >= 6:
        action_last_6 = df_action_of_userid['actionType'].iat[-6]
        actiontime_last_6 = df_action_of_userid['actionTime'].iat[-6]
    if sum_action >= 7:
        action_last_7 = df_action_of_userid['actionType'].iat[-7]
        actiontime_last_7 = df_action_of_userid['actionTime'].iat[-7]
    if sum_action >= 8:
        action_last_8 = df_action_of_userid['actionType'].iat[-8]
        actiontime_last_8 = df_action_of_userid['actionTime'].iat[-8]
    if sum_action >= 9:
        action_last_9 = df_action_of_userid['actionType'].iat[-9]
        actiontime_last_9 = df_action_of_userid['actionTime'].iat[-9]
    if sum_action >= 10:
        action_last_10 = df_action_of_userid['actionType'].iat[-10]
        actiontime_last_10 = df_action_of_userid['actionTime'].iat[-10]
        actiontimespan_mean_1_5 = getActionTimeSpanMean(df_action_of_userid, 1, 5)
        actiontimespan_mean_5_9 = getActionTimeSpanMean(df_action_of_userid, 5, 9)
        actiontimespan_mean_1_9 = getActionTimeSpanMean(df_action_of_userid, 1, 9)
        actiontypeproplast10_1 = np.sum(df_action_of_userid.iloc[-10:]['actionType']==1)
        actiontypeproplast10_2 = np.sum(df_action_of_userid.iloc[-10:]['actionType']==2)
        actiontypeproplast10_3 = np.sum(df_action_of_userid.iloc[-10:]['actionType']==3)
        actiontypeproplast10_4 = np.sum(df_action_of_userid.iloc[-10:]['actionType']==4)
        actiontypeproplast10_5 = np.sum(df_action_of_userid.iloc[-10:]['actionType']==5)
        actiontypeproplast10_6 = np.sum(df_action_of_userid.iloc[-10:]['actionType']==6)
        actiontypeproplast10_7 = np.sum(df_action_of_userid.iloc[-10:]['actionType']==7)
        actiontypeproplast10_8 = np.sum(df_action_of_userid.iloc[-10:]['actionType']==8)
        actiontypeproplast10_9 = np.sum(df_action_of_userid.iloc[-10:]['actionType']==9)
        actiontimespancount_last10_1_5 = getActionTimeSpanCount(df_action_of_userid.iloc[-10:], 1, 5, timethred=50)
        actiontimespancount_last10_5_6 = getActionTimeSpanCount(df_action_of_userid.iloc[-10:], 5, 6, timethred=50)
        actiontimespancount_last10_6_7 = getActionTimeSpanCount(df_action_of_userid.iloc[-10:], 6, 7, timethred=50)
        actiontimespancount_last10_7_8 = getActionTimeSpanCount(df_action_of_userid.iloc[-10:], 7, 8, timethred=50)
        actiontimespancount_last10_8_9 = getActionTimeSpanCount(df_action_of_userid.iloc[-10:], 8, 9, timethred=50)
    if sum_action >= 11:
        action_last_11 = df_action_of_userid['actionType'].iat[-11]
        actiontime_last_11 = df_action_of_userid['actionTime'].iat[-11]
    if sum_action >= 12:
        action_last_12 = df_action_of_userid['actionType'].iat[-12]
        actiontime_last_12 = df_action_of_userid['actionTime'].iat[-12]
    if sum_action >= 13:
        action_last_13 = df_action_of_userid['actionType'].iat[-13]
        actiontime_last_13 = df_action_of_userid['actionTime'].iat[-13]
    if sum_action >= 14:
        action_last_14 = df_action_of_userid['actionType'].iat[-14]
        actiontime_last_14 = df_action_of_userid['actionTime'].iat[-14]
    if sum_action >= 15:
        action_last_15 = df_action_of_userid['actionType'].iat[-15]
        actiontime_last_15 = df_action_of_userid['actionTime'].iat[-15]
    if sum_action >= 16:
        action_last_16 = df_action_of_userid['actionType'].iat[-16]
        actiontime_last_16 = df_action_of_userid['actionTime'].iat[-16]
    if sum_action >= 17:
        action_last_17 = df_action_of_userid['actionType'].iat[-17]
        actiontime_last_17 = df_action_of_userid['actionTime'].iat[-17]
    if sum_action >= 18:
        action_last_18 = df_action_of_userid['actionType'].iat[-18]
        actiontime_last_18 = df_action_of_userid['actionTime'].iat[-18]
    if sum_action >= 19:
        action_last_19 = df_action_of_userid['actionType'].iat[-19]
        actiontime_last_19 = df_action_of_userid['actionTime'].iat[-19]
    if sum_action >= 20:
        action_last_20 = df_action_of_userid['actionType'].iat[-20]
        actiontime_last_20 = df_action_of_userid['actionTime'].iat[-20]
    actiontypeprop_1 = np.sum(df_action_of_userid['actionType']==1) / (sum_action+1.0)
    actiontypeprop_2 = np.sum(df_action_of_userid['actionType']==2) / (sum_action+1.0)
    actiontypeprop_3 = np.sum(df_action_of_userid['actionType']==3) / (sum_action+1.0)
    actiontypeprop_4 = np.sum(df_action_of_userid['actionType']==4) / (sum_action+1.0)
    actiontypeprop_5 = np.sum(df_action_of_userid['actionType']==5) / (sum_action+1.0)
    actiontypeprop_6 = np.sum(df_action_of_userid['actionType']==6) / (sum_action+1.0)
    actiontypeprop_7 = np.sum(df_action_of_userid['actionType']==7) / (sum_action+1.0)
    actiontypeprop_8 = np.sum(df_action_of_userid['actionType']==8) / (sum_action+1.0)
    actiontypeprop_9 = np.sum(df_action_of_userid['actionType']==9) / (sum_action+1.0)
    actiontimespancount_1_5 = getActionTimeSpanCount(df_action_of_userid, 1, 5, timespanthred)
    actiontimespancount_5_6 = getActionTimeSpanCount(df_action_of_userid, 5, 6, timespanthred)
    actiontimespancount_6_7 = getActionTimeSpanCount(df_action_of_userid, 6, 7, timespanthred)
    actiontimespancount_7_8 = getActionTimeSpanCount(df_action_of_userid, 5, 8, timespanthred)
    actiontimespancount_8_9 = getActionTimeSpanCount(df_action_of_userid, 4, 9, timespanthred)
    sum_action_1 = np.sum(df_action_of_userid['actionType'] == 1)
    sum_action_24 = np.sum((df_action_of_userid['actionType'] >= 2) & (df_action_of_userid['actionType'] <= 4))
    sum_action_59 = np.sum((df_action_of_userid['actionType'] >= 5) & (df_action_of_userid['actionType'] <= 9))
    actionratio_24_59 = (sum_action_24 + 1.0) / (sum_action_59 + 1.0)
    actionratio_1_59 = (sum_action_1 + 1.0) / (sum_action_59 + 1.0)
    return sum_action, actiontime_last_1_year, actiontime_last_1_month, action_last_1, action_last_2, action_last_3, action_last_4, action_last_5, action_last_6, action_last_7, action_last_8, action_last_9, action_last_10, action_last_11, action_last_12, action_last_13, action_last_14, action_last_15, action_last_16, action_last_17, action_last_18, action_last_19, action_last_20, actiontime_last_1, actiontime_last_2, actiontime_last_3, actiontime_last_4, actiontime_last_5, actiontime_last_6, actiontime_last_7, actiontime_last_8, actiontime_last_9, actiontime_last_10, actiontime_last_11, actiontime_last_12, actiontime_last_13, actiontime_last_14, actiontime_last_15, actiontime_last_16, actiontime_last_17, actiontime_last_18, actiontime_last_19, actiontime_last_20, actiontypeprop_1, actiontypeprop_2, actiontypeprop_3, actiontypeprop_4, actiontypeprop_5, actiontypeprop_6, actiontypeprop_7, actiontypeprop_8, actiontypeprop_9, actiontimespancount_1_5, actiontimespancount_5_6, actiontimespancount_6_7, actiontimespancount_7_8, actiontimespancount_8_9, actionratio_24_59, actiontype_lasttime_1, actiontype_lasttime_5, actiontype_lasttime_6, actiontype_lasttime_7, actiontype_lasttime_8, actiontype_lasttime_9, actiontype_lasttime_24, actiontimespanlast_1_5, actiontimespanlast_5_6, actiontimespanlast_6_7, actiontimespanlast_7_8, actiontimespanlast_5_7, actiontimespanlast_5_8, action59seqentialratio, actiontypeproplast10_1, actiontypeproplast10_2, actiontypeproplast10_3, actiontypeproplast10_4, actiontypeproplast10_5, actiontypeproplast10_6, actiontypeproplast10_7, actiontypeproplast10_8, actiontypeproplast10_9, actiontime_1, actiontimespancount_last10_1_5, actiontimespancount_last10_5_6, actiontimespancount_last10_6_7, actiontimespancount_last10_7_8, actiontimespancount_last10_8_9, actiontimespan_mean_1_5, actiontimespan_mean_5_9, actiontimespan_mean_1_9, actionratio_1_59, timespan2last_action1_len, timespan2last_action5_len, timespan2last_action6_len, timespan2last_action7_len, timespan2last_action8_len, timespan2last_action9_len, timespan2last_action1_mean, timespan2last_action5_mean, timespan2last_action6_mean, timespan2last_action7_mean, timespan2last_action8_mean, timespan2last_action9_mean, timespan2last_action1_std, timespan2last_action5_std, timespan2last_action6_std, timespan2last_action7_std, timespan2last_action8_std, timespan2last_action9_std

def getTagsFromCommentByUserid(df_comm, userid):
    df_comm_of_userid = df_comm[df_comm['userid'] == userid]
    sum_comm = len(df_comm_of_userid)                    # 一个用户comment的总次数
    rating_ave = 5.0
    rating_min = 5.0
    rating_last = 5.0
    if sum_comm > 0:
        rating_ave = np.mean(df_comm_of_userid['rating'])
        rating_min = np.min(df_comm_of_userid['rating'])
        rating_last = df_comm_of_userid['rating'].iat[-1]
    return rating_ave, rating_min, rating_last

def getTagsFromProfileByUserid(df_profile, userid, provinceset):
    df_profile_of_userid = df_profile[df_profile['userid'] == userid]
    hasgender = 1   # 是否有男女信息
    ismale = 0      # 是否是男
    isfemale = 0    # 是否是女
    hasage = 1
    is60 = 0
    is70 = 0
    is80 = 0
    is90 = 0
    is00 = 0
    province_df = pd.DataFrame()
    for k in range(len(provinceset)):
        province_df[provinceset[k]] = [0]
    for k in range(len(provinceset)):
        if str(df_profile_of_userid['province'].iat[0]) == provinceset[k]:
            province_df[provinceset[k]] = 1
        else:
            province_df[provinceset[k]] = 0
    if df_profile_of_userid['gender'].iat[0] == '男':
        ismale = 1
    elif df_profile_of_userid['gender'].iat[0] == '女':
        isfemale = 1
    else:
        hasgender = 0
    if df_profile_of_userid['age'].iat[0] == '60后':
        is60 = 1
    elif df_profile_of_userid['age'].iat[0] == '70后':
        is70 = 1
    elif df_profile_of_userid['age'].iat[0] == '80后':
        is80 = 1
    elif df_profile_of_userid['age'].iat[0] == '90后':
        is90 = 1
    elif df_profile_of_userid['age'].iat[0] == '00后':
        is00 = 1
    else:
        hasage = 0
    return hasgender, ismale, isfemale, hasage, is60, is70, is80, is90, is00, province_df

In [3]:
### load data
#train_or_test = 'train'
train_or_test = 'test'
df_profile = pd.read_csv('./data_train_test/userProfile_'+train_or_test+'.csv')
df_ordhistory = pd.read_csv('./data_train_test/orderHistory_'+train_or_test+'.csv')
df_action = pd.read_csv('./data_train_test/action_'+train_or_test+'.csv')
df_comm = pd.read_csv('./data_train_test/userComment_'+train_or_test+'.csv')
df_ordfuture = pd.read_csv('./data_train_test/orderFuture_'+train_or_test+'.csv')
usercount = len(df_profile)
df = pd.DataFrame(df_profile['userid'])
#df['futureOrderType'] = df_ordfuture['orderType']

In [4]:
### extract tags of user
# get tags from orderhistory
df_ordhistory_train = pd.read_csv('./data_train_test/orderHistory_train.csv')
countryset = np.array(list(set(df_ordhistory_train['country'])))
cityset = np.array(list(set(df_ordhistory_train['city'])))
sum_all_list = np.zeros(usercount)
sum0_list = np.zeros(usercount)
sum1_list = np.zeros(usercount)
ratio1_0_list = np.zeros(usercount)
time_last_1_list = np.zeros(usercount)
time_last_1_year_list = np.zeros(usercount)
time_last_1_month_list = np.zeros(usercount)
time_last_1_hour_list = np.zeros(usercount)
time_last_1_yday_list = np.zeros(usercount)
time_last_2_list = np.zeros(usercount)
time_last_2_year_list = np.zeros(usercount)
time_last_2_month_list = np.zeros(usercount)
time_last_2_hour_list = np.zeros(usercount)
time_last_2_yday_list = np.zeros(usercount)
time_last_3_list = np.zeros(usercount)
time_last_3_year_list = np.zeros(usercount)
time_last_3_month_list = np.zeros(usercount)
time_last_3_hour_list = np.zeros(usercount)
time_last_3_yday_list = np.zeros(usercount)
#sum_cont1_list = np.zeros(usercount)
#sum_cont2_list = np.zeros(usercount)
#sum_cont3_list = np.zeros(usercount)
#sum_cont4_list = np.zeros(usercount)
#sum_cont5_list = np.zeros(usercount)
#sum_countries_all_dflist = pd.DataFrame()
#sum_cities_all_dflist = pd.DataFrame()
ordtime_mean_list = np.zeros(usercount)
ordtime_min_list = np.zeros(usercount)
ordtime_max_list = np.zeros(usercount)
ordtime_std_list = np.zeros(usercount)
time_ordtype1_last_1_list = np.zeros(usercount)
time_ordtype1_last_2_list = np.zeros(usercount)
ord_last_1_type_list = np.zeros(usercount)
ord_last_2_type_list = np.zeros(usercount)
ord_last_3_type_list = np.zeros(usercount)
continent_last_1_list = []
country_last_1_list = []
city_last_1_list = []
for i in range(usercount):
    sum_all, sum0, sum1, ratio1_0, time_last_1, time_last_1_year, time_last_1_month, time_last_1_hour, time_last_1_yday, time_last_2, time_last_2_year, time_last_2_month, time_last_2_hour, time_last_2_yday, time_last_3, time_last_3_year, time_last_3_month, time_last_3_hour, time_last_3_yday, ordtime_mean, ordtime_min, ordtime_max, ordtime_std, time_ordtype1_last_1, time_ordtype1_last_2, ord_last_1_type, ord_last_2_type, ord_last_3_type, continent_last_1, country_last_1, city_last_1 = getTagsFromOrderHistoryByUserid(df_ordhistory, df_profile['userid'][i])
    sum_all_list[i] = sum_all
    sum0_list[i] = sum0
    sum1_list[i] = sum1
    ratio1_0_list[i] = ratio1_0
    time_last_1_list[i] = time_last_1
    time_last_1_year_list[i] = time_last_1_year
    time_last_1_month_list[i] = time_last_1_month
    time_last_1_hour_list[i] = time_last_1_hour
    time_last_1_yday_list[i] = time_last_1_yday
    time_last_2_list[i] = time_last_2
    time_last_2_year_list[i] = time_last_2_year
    time_last_2_month_list[i] = time_last_2_month
    time_last_2_hour_list[i] = time_last_2_hour
    time_last_2_yday_list[i] = time_last_2_yday
    time_last_3_list[i] = time_last_3
    time_last_3_year_list[i] = time_last_3_year
    time_last_3_month_list[i] = time_last_3_month
    time_last_3_hour_list[i] = time_last_3_hour
    time_last_3_yday_list[i] = time_last_3_yday
    #sum_cont1_list[i] = sum_cont1
    #sum_cont2_list[i] = sum_cont2
    #sum_cont3_list[i] = sum_cont3
    #sum_cont4_list[i] = sum_cont4
    #sum_cont5_list[i] = sum_cont5
    #sum_countries_all_dflist = sum_countries_all_dflist.append(sum_countries)
    #sum_cities_all_dflist = sum_cities_all_dflist.append(sum_cities)
    ordtime_mean_list[i] = ordtime_mean
    ordtime_min_list[i] = ordtime_min
    ordtime_max_list[i] = ordtime_max
    ordtime_std_list[i] = ordtime_std
    time_ordtype1_last_1_list[i] = time_ordtype1_last_1
    time_ordtype1_last_2_list[i] = time_ordtype1_last_2
    ord_last_1_type_list[i] = ord_last_1_type
    ord_last_2_type_list[i] = ord_last_2_type
    ord_last_3_type_list[i] = ord_last_3_type
    continent_last_1_list.append(continent_last_1)
    country_last_1_list.append(country_last_1)
    city_last_1_list.append(city_last_1)
df['histord_sum_all'] = sum_all_list
df['histord_sum_0'] = sum0_list
df['histord_sum_1'] = sum1_list
df['histord_ratio1_0'] = ratio1_0_list
df['histord_time_last_1'] = time_last_1_list
df['histord_time_last_1_year'] = time_last_1_year_list
df['histord_time_last_1_month'] = time_last_1_month_list
df['histord_time_last_1_hour'] = time_last_1_hour_list
df['histord_time_last_1_yday'] = time_last_1_yday_list
df['histord_time_last_2'] = time_last_2_list
df['histord_time_last_2_year'] = time_last_2_year_list
df['histord_time_last_2_month'] = time_last_2_month_list
df['histord_time_last_2_hour'] = time_last_2_hour_list
df['histord_time_last_2_yday'] = time_last_2_yday_list
df['histord_time_last_3'] = time_last_3_list
df['histord_time_last_3_year'] = time_last_3_year_list
df['histord_time_last_3_month'] = time_last_3_month_list
df['histord_time_last_3_hour'] = time_last_3_hour_list
df['histord_time_last_3_yday'] = time_last_3_yday_list
#df['histord_sum_cont1'] = sum_cont1_list
#df['histord_sum_cont2'] = sum_cont2_list
#df['histord_sum_cont3'] = sum_cont3_list
#df['histord_sum_cont4'] = sum_cont4_list
#df['histord_sum_cont5'] = sum_cont5_list
#sum_countries_all_dflist['id'] = range(0, usercount)
#sum_countries_all_dflist = sum_countries_all_dflist.set_index('id')
#df = pd.concat([df, sum_countries_all_dflist], axis=1)
#sum_cities_all_dflist['id'] = range(0, usercount)
#sum_cities_all_dflist = sum_cities_all_dflist.set_index('id')
#df = pd.concat([df, sum_cities_all_dflist], axis=1)
df['histordtime_mean'] = ordtime_mean_list
df['histordtime_min'] = ordtime_min_list
df['histordtime_max'] = ordtime_max_list
df['histordtime_std'] = ordtime_std_list
df['histord_time_ordtype1_last_1'] = time_ordtype1_last_1_list
df['histord_time_ordtype1_last_2'] = time_ordtype1_last_2_list
df['ord_last_1_type'] = ord_last_1_type_list
df['ord_last_2_type'] = ord_last_2_type_list
df['ord_last_3_type'] = ord_last_3_type_list
#df['continent_last_1'] = continent_last_1_list
#df['country_last_1'] = country_last_1_list
#df['city_last_1'] = city_last_1_list
df = pd.concat([df, pd.get_dummies(continent_last_1_list)], axis=1)
df = pd.concat([df, pd.get_dummies(country_last_1_list)], axis=1)
df = pd.concat([df, pd.get_dummies(city_last_1_list)], axis=1)

In [5]:
# get tags from action
sum_action_list = np.zeros(usercount)
actiontime_last_1_year_list = np.zeros(usercount)
actiontime_last_1_month_list = np.zeros(usercount)
action_last_1_list = np.zeros(usercount)
action_last_2_list = np.zeros(usercount)
action_last_3_list = np.zeros(usercount)
action_last_4_list = np.zeros(usercount)
action_last_5_list = np.zeros(usercount)
action_last_6_list = np.zeros(usercount)
action_last_7_list = np.zeros(usercount)
action_last_8_list = np.zeros(usercount)
action_last_9_list = np.zeros(usercount)
action_last_10_list = np.zeros(usercount)
action_last_11_list = np.zeros(usercount)
action_last_12_list = np.zeros(usercount)
action_last_13_list = np.zeros(usercount)
action_last_14_list = np.zeros(usercount)
action_last_15_list = np.zeros(usercount)
action_last_16_list = np.zeros(usercount)
action_last_17_list = np.zeros(usercount)
action_last_18_list = np.zeros(usercount)
action_last_19_list = np.zeros(usercount)
action_last_20_list = np.zeros(usercount)
actiontime_last_1_list = np.zeros(usercount)
actiontime_last_2_list = np.zeros(usercount)
actiontime_last_3_list = np.zeros(usercount)
actiontime_last_4_list = np.zeros(usercount)
actiontime_last_5_list = np.zeros(usercount)
actiontime_last_6_list = np.zeros(usercount)
actiontime_last_7_list = np.zeros(usercount)
actiontime_last_8_list = np.zeros(usercount)
actiontime_last_9_list = np.zeros(usercount)
actiontime_last_10_list = np.zeros(usercount)
actiontime_last_11_list = np.zeros(usercount)
actiontime_last_12_list = np.zeros(usercount)
actiontime_last_13_list = np.zeros(usercount)
actiontime_last_14_list = np.zeros(usercount)
actiontime_last_15_list = np.zeros(usercount)
actiontime_last_16_list = np.zeros(usercount)
actiontime_last_17_list = np.zeros(usercount)
actiontime_last_18_list = np.zeros(usercount)
actiontime_last_19_list = np.zeros(usercount)
actiontime_last_20_list = np.zeros(usercount)
actiontypeprop_1_list = np.zeros(usercount)
actiontypeprop_2_list = np.zeros(usercount)
actiontypeprop_3_list = np.zeros(usercount)
actiontypeprop_4_list = np.zeros(usercount)
actiontypeprop_5_list = np.zeros(usercount)
actiontypeprop_6_list = np.zeros(usercount)
actiontypeprop_7_list = np.zeros(usercount)
actiontypeprop_8_list = np.zeros(usercount)
actiontypeprop_9_list = np.zeros(usercount)
actiontimespancount_1_5_list = np.zeros(usercount)
actiontimespancount_5_6_list = np.zeros(usercount)
actiontimespancount_6_7_list = np.zeros(usercount)
actiontimespancount_7_8_list = np.zeros(usercount)
actiontimespancount_8_9_list = np.zeros(usercount)
actionratio_24_59_list = np.zeros(usercount)
actiontype_lasttime_1_list = np.zeros(usercount)
actiontype_lasttime_5_list = np.zeros(usercount)
actiontype_lasttime_6_list = np.zeros(usercount)
actiontype_lasttime_7_list = np.zeros(usercount)
actiontype_lasttime_8_list = np.zeros(usercount)
actiontype_lasttime_9_list = np.zeros(usercount)
actiontype_lasttime_24_list = np.zeros(usercount)
actiontimespanlast_1_5_list = np.zeros(usercount)
actiontimespanlast_5_6_list = np.zeros(usercount)
actiontimespanlast_6_7_list = np.zeros(usercount)
actiontimespanlast_7_8_list = np.zeros(usercount)
actiontimespanlast_5_7_list = np.zeros(usercount)
actiontimespanlast_5_8_list = np.zeros(usercount)
action59seqentialratio_list = np.zeros(usercount)
actiontypeproplast10_1_list = np.zeros(usercount)
actiontypeproplast10_2_list = np.zeros(usercount)
actiontypeproplast10_3_list = np.zeros(usercount)
actiontypeproplast10_4_list = np.zeros(usercount)
actiontypeproplast10_5_list = np.zeros(usercount)
actiontypeproplast10_6_list = np.zeros(usercount)
actiontypeproplast10_7_list = np.zeros(usercount)
actiontypeproplast10_8_list = np.zeros(usercount)
actiontypeproplast10_9_list = np.zeros(usercount)
actiontime_1_list = np.zeros(usercount)
actiontimespancount_last10_1_5_list = np.zeros(usercount)
actiontimespancount_last10_5_6_list = np.zeros(usercount)
actiontimespancount_last10_6_7_list = np.zeros(usercount)
actiontimespancount_last10_7_8_list = np.zeros(usercount)
actiontimespancount_last10_8_9_list = np.zeros(usercount)
actiontimespan_mean_1_5_list = np.zeros(usercount)
actiontimespan_mean_5_9_list = np.zeros(usercount)
actiontimespan_mean_1_9_list = np.zeros(usercount)
actionratio_1_59_list = np.zeros(usercount)
timespan2last_action1_len_list = np.zeros(usercount)
timespan2last_action5_len_list = np.zeros(usercount)
timespan2last_action6_len_list = np.zeros(usercount)
timespan2last_action7_len_list = np.zeros(usercount)
timespan2last_action8_len_list = np.zeros(usercount)
timespan2last_action9_len_list = np.zeros(usercount)
timespan2last_action1_mean_list = np.zeros(usercount)
timespan2last_action5_mean_list = np.zeros(usercount)
timespan2last_action6_mean_list = np.zeros(usercount)
timespan2last_action7_mean_list = np.zeros(usercount)
timespan2last_action8_mean_list = np.zeros(usercount)
timespan2last_action9_mean_list = np.zeros(usercount)
timespan2last_action1_std_list = np.zeros(usercount)
timespan2last_action5_std_list = np.zeros(usercount)
timespan2last_action6_std_list = np.zeros(usercount)
timespan2last_action7_std_list = np.zeros(usercount)
timespan2last_action8_std_list = np.zeros(usercount)
timespan2last_action9_std_list = np.zeros(usercount)
for i in range(usercount):
    sum_action, actiontime_last_1_year, actiontime_last_1_month, action_last_1, action_last_2, action_last_3, action_last_4, action_last_5, action_last_6, action_last_7, action_last_8, action_last_9, action_last_10, action_last_11, action_last_12, action_last_13, action_last_14, action_last_15, action_last_16, action_last_17, action_last_18, action_last_19, action_last_20, actiontime_last_1, actiontime_last_2, actiontime_last_3, actiontime_last_4, actiontime_last_5, actiontime_last_6, actiontime_last_7, actiontime_last_8, actiontime_last_9, actiontime_last_10, actiontime_last_11, actiontime_last_12, actiontime_last_13, actiontime_last_14, actiontime_last_15, actiontime_last_16, actiontime_last_17, actiontime_last_18, actiontime_last_19, actiontime_last_20, actiontypeprop_1, actiontypeprop_2, actiontypeprop_3, actiontypeprop_4, actiontypeprop_5, actiontypeprop_6, actiontypeprop_7, actiontypeprop_8, actiontypeprop_9, actiontimespancount_1_5, actiontimespancount_5_6, actiontimespancount_6_7, actiontimespancount_7_8, actiontimespancount_8_9, actionratio_24_59, actiontype_lasttime_1, actiontype_lasttime_5, actiontype_lasttime_6, actiontype_lasttime_7, actiontype_lasttime_8, actiontype_lasttime_9, actiontype_lasttime_24, actiontimespanlast_1_5, actiontimespanlast_5_6, actiontimespanlast_6_7, actiontimespanlast_7_8, actiontimespanlast_5_7, actiontimespanlast_5_8, action59seqentialratio, actiontypeproplast10_1, actiontypeproplast10_2, actiontypeproplast10_3, actiontypeproplast10_4, actiontypeproplast10_5, actiontypeproplast10_6, actiontypeproplast10_7, actiontypeproplast10_8, actiontypeproplast10_9, actiontime_1, actiontimespancount_last10_1_5, actiontimespancount_last10_5_6, actiontimespancount_last10_6_7, actiontimespancount_last10_7_8, actiontimespancount_last10_8_9, actiontimespan_mean_1_5, actiontimespan_mean_5_9, actiontimespan_mean_1_9, actionratio_1_59, timespan2last_action1_len, timespan2last_action5_len, timespan2last_action6_len, timespan2last_action7_len, timespan2last_action8_len, timespan2last_action9_len, timespan2last_action1_mean, timespan2last_action5_mean, timespan2last_action6_mean, timespan2last_action7_mean, timespan2last_action8_mean, timespan2last_action9_mean, timespan2last_action1_std, timespan2last_action5_std, timespan2last_action6_std, timespan2last_action7_std, timespan2last_action8_std, timespan2last_action9_std = getTagsFromActionByUserid(df_action, df_profile['userid'][i])
    sum_action_list[i] = sum_action
    actiontime_last_1_year_list[i] = actiontime_last_1_year
    actiontime_last_1_month_list[i] = actiontime_last_1_month
    action_last_1_list[i] = action_last_1
    action_last_2_list[i] = action_last_2
    action_last_3_list[i] = action_last_3
    action_last_4_list[i] = action_last_4
    action_last_5_list[i] = action_last_5
    action_last_6_list[i] = action_last_6
    action_last_7_list[i] = action_last_7
    action_last_8_list[i] = action_last_8
    action_last_9_list[i] = action_last_9
    action_last_10_list[i] = action_last_10
    action_last_11_list[i] = action_last_11
    action_last_12_list[i] = action_last_12
    action_last_13_list[i] = action_last_13
    action_last_14_list[i] = action_last_14
    action_last_15_list[i] = action_last_15
    action_last_16_list[i] = action_last_16
    action_last_17_list[i] = action_last_17
    action_last_18_list[i] = action_last_18
    action_last_19_list[i] = action_last_19
    action_last_20_list[i] = action_last_20
    actiontime_last_1_list[i] = actiontime_last_1
    actiontime_last_2_list[i] = actiontime_last_2
    actiontime_last_3_list[i] = actiontime_last_3
    actiontime_last_4_list[i] = actiontime_last_4
    actiontime_last_5_list[i] = actiontime_last_5
    actiontime_last_6_list[i] = actiontime_last_6
    actiontime_last_7_list[i] = actiontime_last_7
    actiontime_last_8_list[i] = actiontime_last_8
    actiontime_last_9_list[i] = actiontime_last_9
    actiontime_last_10_list[i] = actiontime_last_10
    actiontime_last_11_list[i] = actiontime_last_11
    actiontime_last_12_list[i] = actiontime_last_12
    actiontime_last_13_list[i] = actiontime_last_13
    actiontime_last_14_list[i] = actiontime_last_14
    actiontime_last_15_list[i] = actiontime_last_15
    actiontime_last_16_list[i] = actiontime_last_16
    actiontime_last_17_list[i] = actiontime_last_17
    actiontime_last_18_list[i] = actiontime_last_18
    actiontime_last_19_list[i] = actiontime_last_19
    actiontime_last_20_list[i] = actiontime_last_20
    actiontypeprop_1_list[i] = actiontypeprop_1
    actiontypeprop_2_list[i] = actiontypeprop_2
    actiontypeprop_3_list[i] = actiontypeprop_3
    actiontypeprop_4_list[i] = actiontypeprop_4
    actiontypeprop_5_list[i] = actiontypeprop_5
    actiontypeprop_6_list[i] = actiontypeprop_6
    actiontypeprop_7_list[i] = actiontypeprop_7
    actiontypeprop_8_list[i] = actiontypeprop_8
    actiontypeprop_9_list[i] = actiontypeprop_9
    actiontimespancount_1_5_list[i] = actiontimespancount_1_5
    actiontimespancount_5_6_list[i] = actiontimespancount_5_6
    actiontimespancount_6_7_list[i] = actiontimespancount_6_7
    actiontimespancount_7_8_list[i] = actiontimespancount_7_8
    actiontimespancount_8_9_list[i] = actiontimespancount_8_9
    actionratio_24_59_list[i] = actionratio_24_59
    actiontype_lasttime_1_list[i] = actiontype_lasttime_1
    actiontype_lasttime_5_list[i] = actiontype_lasttime_5
    actiontype_lasttime_6_list[i] = actiontype_lasttime_6
    actiontype_lasttime_7_list[i] = actiontype_lasttime_7
    actiontype_lasttime_8_list[i] = actiontype_lasttime_8
    actiontype_lasttime_9_list[i] = actiontype_lasttime_9
    actiontype_lasttime_24_list[i] = actiontype_lasttime_24
    actiontimespanlast_1_5_list[i] = actiontimespanlast_1_5
    actiontimespanlast_5_6_list[i] = actiontimespanlast_5_6
    actiontimespanlast_6_7_list[i] = actiontimespanlast_6_7
    actiontimespanlast_7_8_list[i] = actiontimespanlast_7_8
    actiontimespanlast_5_7_list[i] = actiontimespanlast_5_7
    actiontimespanlast_5_8_list[i] = actiontimespanlast_5_8
    action59seqentialratio_list[i] = action59seqentialratio
    actiontypeproplast10_1_list[i] = actiontypeproplast10_1
    actiontypeproplast10_2_list[i] = actiontypeproplast10_2
    actiontypeproplast10_3_list[i] = actiontypeproplast10_3
    actiontypeproplast10_4_list[i] = actiontypeproplast10_4
    actiontypeproplast10_5_list[i] = actiontypeproplast10_5
    actiontypeproplast10_6_list[i] = actiontypeproplast10_6
    actiontypeproplast10_7_list[i] = actiontypeproplast10_7
    actiontypeproplast10_8_list[i] = actiontypeproplast10_8
    actiontypeproplast10_9_list[i] = actiontypeproplast10_9
    actiontime_1_list[i] = actiontime_1
    actiontimespancount_last10_1_5_list[i] = actiontimespancount_last10_1_5
    actiontimespancount_last10_5_6_list[i] = actiontimespancount_last10_5_6
    actiontimespancount_last10_6_7_list[i] = actiontimespancount_last10_6_7
    actiontimespancount_last10_7_8_list[i] = actiontimespancount_last10_7_8
    actiontimespancount_last10_8_9_list[i] = actiontimespancount_last10_8_9
    actiontimespan_mean_1_5_list[i] = actiontimespan_mean_1_5
    actiontimespan_mean_5_9_list[i] = actiontimespan_mean_5_9
    actiontimespan_mean_1_9_list[i] = actiontimespan_mean_1_9
    actionratio_1_59_list[i] = actionratio_1_59
    timespan2last_action1_len_list[i] = timespan2last_action1_len
    timespan2last_action5_len_list[i] = timespan2last_action5_len
    timespan2last_action6_len_list[i] = timespan2last_action6_len
    timespan2last_action7_len_list[i] = timespan2last_action7_len
    timespan2last_action8_len_list[i] = timespan2last_action8_len
    timespan2last_action9_len_list[i] = timespan2last_action9_len
    timespan2last_action1_mean_list[i] = timespan2last_action1_mean
    timespan2last_action5_mean_list[i] = timespan2last_action5_mean
    timespan2last_action6_mean_list[i] = timespan2last_action6_mean
    timespan2last_action7_mean_list[i] = timespan2last_action7_mean
    timespan2last_action8_mean_list[i] = timespan2last_action8_mean
    timespan2last_action9_mean_list[i] = timespan2last_action9_mean
    timespan2last_action1_std_list[i] = timespan2last_action1_std
    timespan2last_action5_std_list[i] = timespan2last_action5_std
    timespan2last_action6_std_list[i] = timespan2last_action6_std
    timespan2last_action7_std_list[i] = timespan2last_action7_std
    timespan2last_action8_std_list[i] = timespan2last_action8_std
    timespan2last_action9_std_list[i] = timespan2last_action9_std
df['action_sum'] = sum_action_list
df['actiontime_last_1_year'] = actiontime_last_1_year_list
df['actiontime_last_1_month'] = actiontime_last_1_month_list
df['actiontype_last_1'] = action_last_1_list
df['actiontype_last_2'] = action_last_2_list
df['actiontype_last_3'] = action_last_3_list
df['actiontype_last_4'] = action_last_4_list
df['actiontype_last_5'] = action_last_5_list
df['actiontype_last_6'] = action_last_6_list
df['actiontype_last_7'] = action_last_7_list
df['actiontype_last_8'] = action_last_8_list
df['actiontype_last_9'] = action_last_9_list
df['actiontype_last_10'] = action_last_10_list
df['actiontype_last_11'] = action_last_11_list
df['actiontype_last_12'] = action_last_12_list
df['actiontype_last_13'] = action_last_13_list
df['actiontype_last_14'] = action_last_14_list
df['actiontype_last_15'] = action_last_15_list
df['actiontype_last_16'] = action_last_16_list
df['actiontype_last_17'] = action_last_17_list
df['actiontype_last_18'] = action_last_18_list
df['actiontype_last_19'] = action_last_19_list
df['actiontype_last_20'] = action_last_20_list
df['actiontime_last_1'] = actiontime_last_1_list
df['actiontime_last_2'] = actiontime_last_2_list
df['actiontime_last_3'] = actiontime_last_3_list
df['actiontime_last_4'] = actiontime_last_4_list
df['actiontime_last_5'] = actiontime_last_5_list
df['actiontime_last_6'] = actiontime_last_6_list
df['actiontime_last_7'] = actiontime_last_7_list
df['actiontime_last_8'] = actiontime_last_8_list
df['actiontime_last_9'] = actiontime_last_9_list
df['actiontime_last_10'] = actiontime_last_10_list
df['actiontime_last_11'] = actiontime_last_11_list
df['actiontime_last_12'] = actiontime_last_12_list
df['actiontime_last_13'] = actiontime_last_13_list
df['actiontime_last_14'] = actiontime_last_14_list
df['actiontime_last_15'] = actiontime_last_15_list
df['actiontime_last_16'] = actiontime_last_16_list
df['actiontime_last_17'] = actiontime_last_17_list
df['actiontime_last_18'] = actiontime_last_18_list
df['actiontime_last_19'] = actiontime_last_19_list
df['actiontime_last_20'] = actiontime_last_20_list
df['actiontypeprop_1'] = actiontypeprop_1_list
df['actiontypeprop_2'] = actiontypeprop_2_list
df['actiontypeprop_3'] = actiontypeprop_3_list
df['actiontypeprop_4'] = actiontypeprop_4_list
df['actiontypeprop_5'] = actiontypeprop_5_list
df['actiontypeprop_6'] = actiontypeprop_6_list
df['actiontypeprop_7'] = actiontypeprop_7_list
df['actiontypeprop_8'] = actiontypeprop_8_list
df['actiontypeprop_9'] = actiontypeprop_9_list
df['actiontimespancount_1_5'] = actiontimespancount_1_5_list
df['actiontimespancount_5_6'] = actiontimespancount_5_6_list
df['actiontimespancount_6_7'] = actiontimespancount_6_7_list
df['actiontimespancount_7_8'] = actiontimespancount_7_8_list
df['actiontimespancount_8_9'] = actiontimespancount_8_9_list
df['actionratio_24_59'] = actionratio_24_59_list
df['actiontype_lasttime_1'] = actiontype_lasttime_1_list
df['actiontype_lasttime_5'] = actiontype_lasttime_5_list
df['actiontype_lasttime_6'] = actiontype_lasttime_6_list
df['actiontype_lasttime_7'] = actiontype_lasttime_7_list
df['actiontype_lasttime_8'] = actiontype_lasttime_8_list
df['actiontype_lasttime_9'] = actiontype_lasttime_9_list
df['actiontype_lasttime_24'] = actiontype_lasttime_24_list
df['actiontimespanlast_1_5'] = actiontimespanlast_1_5_list
df['actiontimespanlast_5_6'] = actiontimespanlast_5_6_list
df['actiontimespanlast_6_7'] = actiontimespanlast_6_7_list
df['actiontimespanlast_7_8'] = actiontimespanlast_7_8_list
df['actiontimespanlast_5_7'] = actiontimespanlast_5_7_list
df['actiontimespanlast_5_8'] = actiontimespanlast_5_8_list
df['action59seqentialratio'] = action59seqentialratio_list
df['actiontypeproplast10_1'] = actiontypeproplast10_1_list
df['actiontypeproplast10_2'] = actiontypeproplast10_2_list
df['actiontypeproplast10_3'] = actiontypeproplast10_3_list
df['actiontypeproplast10_4'] = actiontypeproplast10_4_list
df['actiontypeproplast10_5'] = actiontypeproplast10_5_list
df['actiontypeproplast10_6'] = actiontypeproplast10_6_list
df['actiontypeproplast10_7'] = actiontypeproplast10_7_list
df['actiontypeproplast10_8'] = actiontypeproplast10_8_list
df['actiontypeproplast10_9'] = actiontypeproplast10_9_list
df['actiontime_1'] = actiontime_1_list
df['actiontimespancount_last10_1_5'] = actiontimespancount_last10_1_5_list
df['actiontimespancount_last10_5_6'] = actiontimespancount_last10_5_6_list
df['actiontimespancount_last10_6_7'] = actiontimespancount_last10_6_7_list
df['actiontimespancount_last10_7_8'] = actiontimespancount_last10_7_8_list
df['actiontimespancount_last10_8_9'] = actiontimespancount_last10_8_9_list
df['actiontimespan_mean_1_5'] = actiontimespan_mean_1_5_list
df['actiontimespan_mean_5_9'] = actiontimespan_mean_5_9_list
df['actiontimespan_mean_1_9'] = actiontimespan_mean_1_9_list
df['actionratio_1_59'] = actionratio_1_59_list
df['timespan2last_action1_len'] = timespan2last_action1_len_list
df['timespan2last_action5_len'] = timespan2last_action5_len_list
df['timespan2last_action6_len'] = timespan2last_action6_len_list
df['timespan2last_action7_len'] = timespan2last_action7_len_list
df['timespan2last_action8_len'] = timespan2last_action8_len_list
df['timespan2last_action9_len'] = timespan2last_action9_len_list
df['timespan2last_action1_mean'] = timespan2last_action1_mean_list
df['timespan2last_action5_mean'] = timespan2last_action5_mean_list
df['timespan2last_action6_mean'] = timespan2last_action6_mean_list
df['timespan2last_action7_mean'] = timespan2last_action7_mean_list
df['timespan2last_action8_mean'] = timespan2last_action8_mean_list
df['timespan2last_action9_mean'] = timespan2last_action9_mean_list
df['timespan2last_action1_std'] = timespan2last_action1_std_list
df['timespan2last_action5_std'] = timespan2last_action5_std_list
df['timespan2last_action6_std'] = timespan2last_action6_std_list
df['timespan2last_action7_std'] = timespan2last_action7_std_list
df['timespan2last_action8_std'] = timespan2last_action8_std_list
df['timespan2last_action9_std'] = timespan2last_action9_std_list

In [6]:
# get tags from comment
rating_ave_list = np.zeros(usercount)
rating_min_list = np.zeros(usercount)
rating_last_list = np.zeros(usercount)
for i in range(usercount):
    rating_ave, rating_min, rating_last = getTagsFromCommentByUserid(df_comm, df_profile['userid'][i])
    rating_ave_list[i] = rating_ave
    rating_min_list[i] = rating_min
    rating_last_list[i] = rating_last
df['rating_mean'] = rating_ave_list
df['rating_min'] = rating_min_list
df['rating_last'] = rating_last_list

In [7]:
# get tags from profile
df_userprofile_train = pd.read_csv('./data_train_test/userProfile_train.csv')
provinceset = np.array(list(set(df_userprofile_train['province'])))
hasgender_list = np.zeros(usercount)
ismale_list = np.zeros(usercount)
isfemale_list = np.zeros(usercount)
hasage_list = np.zeros(usercount)
is60_list = np.zeros(usercount)
is70_list = np.zeros(usercount)
is80_list = np.zeros(usercount)
is90_list = np.zeros(usercount)
is00_list = np.zeros(usercount)
province_all_dflist = pd.DataFrame()
for i in range(usercount):
    hasgender, ismale, isfemale, hasage, is60, is70, is80, is90, is00, province_df = getTagsFromProfileByUserid(df_profile, df_profile['userid'][i], provinceset)
    hasgender_list[i] = hasgender
    ismale_list[i] = ismale
    isfemale_list[i] = isfemale
    hasage_list[i] = hasage
    is60_list[i] = is60
    is70_list[i] = is70
    is80_list[i] = is80
    is90_list[i] = is90
    is00_list[i] = is00
    province_all_dflist = province_all_dflist.append(province_df)
df['gender_exist'] = hasgender_list
df['gender_male'] = ismale_list
df['gender_female'] = isfemale_list
df['age_exist'] = hasage_list
df['age_60'] = is60_list
df['age_70'] = is70_list
df['age_80'] = is80_list
df['age_90'] = is90_list
df['age_00'] = is00_list
province_all_dflist['id'] = range(0, usercount)
province_all_dflist = province_all_dflist.set_index('id')
province_all_dflist.rename(columns={'nan': 'hasprovince'}, inplace=True)
df = pd.concat([df, province_all_dflist], axis=1)

In [8]:
# get other information between different tables
#df = pd.read_csv('./result/data_train.csv', encoding='gb2312')
#df = pd.read_csv('./result/data_test.csv', encoding='gb2312')
df['timespan_action_lastord'] = df['actiontime_last_1'] - df['histord_time_last_1']
df['timespan_action1tolast'] = df['actiontime_last_1'] - df['actiontype_lasttime_1']
df['timespan_action5tolast'] = df['actiontime_last_1'] - df['actiontype_lasttime_5']
df['timespan_action6tolast'] = df['actiontime_last_1'] - df['actiontype_lasttime_6']
df['timespan_action7tolast'] = df['actiontime_last_1'] - df['actiontype_lasttime_7']
df['timespan_action8tolast'] = df['actiontime_last_1'] - df['actiontype_lasttime_8']
df['timespan_action9tolast'] = df['actiontime_last_1'] - df['actiontype_lasttime_9']
df['timespan_action24tolast'] = df['actiontime_last_1'] -  df['actiontype_lasttime_24']
df['timespan_lastord_1_2'] = df['histord_time_last_1'] - df['histord_time_last_2']
df['timespan_lastord_2_3'] = df['histord_time_last_2'] - df['histord_time_last_3']
df['timespan_last_1'] = (df['actiontime_last_1'] - df['actiontime_last_2'])
df['timespan_last_2'] = (df['actiontime_last_2'] - df['actiontime_last_3'])
df['timespan_last_3'] = (df['actiontime_last_3'] - df['actiontime_last_4'])
df['timespan_last_4'] = (df['actiontime_last_4'] - df['actiontime_last_5'])
df['timespan_last_5'] = (df['actiontime_last_5'] - df['actiontime_last_6'])
df['timespan_last_6'] = (df['actiontime_last_6'] - df['actiontime_last_7'])
df['timespan_last_7'] = (df['actiontime_last_7'] - df['actiontime_last_8'])
df['timespan_last_8'] = (df['actiontime_last_8'] - df['actiontime_last_9'])
df['timespan_last_9'] = (df['actiontime_last_9'] - df['actiontime_last_10'])
df['timespanmean_last_3'] = np.mean(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3']], axis=1)
df['timespanmin_last_3'] = np.min(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3']], axis=1)
df['timespanmax_last_3'] = np.max(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3']], axis=1)
df['timespanstd_last_3'] = np.std(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3']], axis=1)
df['timespanmean_last_4'] = np.mean(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4']], axis=1)
df['timespanmin_last_4'] = np.min(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4']], axis=1)
df['timespanmax_last_4'] = np.max(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4']], axis=1)
df['timespanstd_last_4'] = np.std(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4']], axis=1)
df['timespanmean_last_5'] = np.mean(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5']], axis=1)
df['timespanmin_last_5'] = np.min(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5']], axis=1)
df['timespanmax_last_5'] = np.max(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5']], axis=1)
df['timespanstd_last_5'] = np.std(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5']], axis=1)
df['timespanmean_last_6'] = np.mean(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6']], axis=1)
df['timespanmin_last_6'] = np.min(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6']], axis=1)
df['timespanmax_last_6'] = np.max(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6']], axis=1)
df['timespanstd_last_6'] = np.std(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6']], axis=1)
df['timespanmean_last_7'] = np.mean(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6', 'timespan_last_7']], axis=1)
df['timespanmin_last_7'] = np.min(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6', 'timespan_last_7']], axis=1)
df['timespanmax_last_7'] = np.max(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6', 'timespan_last_7']], axis=1)
df['timespanstd_last_7'] = np.std(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6', 'timespan_last_7']], axis=1)
df['timespanmean_last_8'] = np.mean(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6', 'timespan_last_7', 'timespan_last_8']], axis=1)
df['timespanmin_last_8'] = np.min(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6', 'timespan_last_7', 'timespan_last_8']], axis=1)
df['timespanmax_last_8'] = np.max(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6', 'timespan_last_7', 'timespan_last_8']], axis=1)
df['timespanstd_last_8'] = np.std(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6', 'timespan_last_7', 'timespan_last_8']], axis=1)
df['timespanmean_last_9'] = np.mean(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6', 'timespan_last_7', 'timespan_last_8', 'timespan_last_9']], axis=1)
df['timespanmin_last_9'] = np.min(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6', 'timespan_last_7', 'timespan_last_8', 'timespan_last_9']], axis=1)
df['timespanmax_last_9'] = np.max(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6', 'timespan_last_7', 'timespan_last_8', 'timespan_last_9']], axis=1)
df['timespanstd_last_9'] = np.std(df[['timespan_last_1', 'timespan_last_2', 'timespan_last_3', 'timespan_last_4', 'timespan_last_5', 'timespan_last_6', 'timespan_last_7', 'timespan_last_8', 'timespan_last_9']], axis=1)
df['timespan_total'] = df['actiontime_last_1'] - df['actiontime_1']
df['timespan_action_lastordtype1'] = df['actiontime_last_1'] - df['histord_time_ordtype1_last_1']
df['timespan_lastordtype1_1_2'] = df['histord_time_ordtype1_last_1'] - df['histord_time_ordtype1_last_2']
df['timespan_lastord_lastordtype1'] = df['histord_time_last_1'] - df['histord_time_ordtype1_last_1']

yday_holi_list = [1, 40, 121, 182]
actiontime_last_1_month_list = np.zeros(len(df))
actiontime_last_1_hour_list = np.zeros(len(df))
actiontime_last_1_yday_list = np.zeros(len(df))
actiontime_last_1_timespan2holi_list = np.zeros(len(df))
for i in range(len(df)):
    actiontime_last1 = df['actiontime_last_1'].iloc[i]
    tm_year, tm_mon, tm_day, tm_hour, tm_min, tm_sec, tm_wday, tm_yday = getTimeFromTimestamp(int(actiontime_last1))
    actiontime_last_1_month_list[i] = tm_mon
    actiontime_last_1_hour_list[i] = tm_hour
    actiontime_last_1_yday_list[i] = tm_yday
    actiontime_last_1_timespan2holi_list[i] = getTimespanWithHolidays(tm_yday, yday_holi_list)
df['actiontime_last_1_month'] = actiontime_last_1_month_list
df['actiontime_last_1_hour'] = actiontime_last_1_hour_list
df['actiontime_last_1_yday'] = actiontime_last_1_yday_list
df['actiontime_last_1_timespan2holi'] = actiontime_last_1_timespan2holi_list

histord_time_ordtype1_last_1_month_list = np.zeros(len(df))
histord_time_ordtype1_last_1_hour_list = np.zeros(len(df))
histord_time_ordtype1_last_1_yday_list = np.zeros(len(df))
histord_time_ordtype1_last_1_timespan2holi_list = np.zeros(len(df))
for i in range(len(df)):
    histord_time_ordtype1_last_1 = df['histord_time_ordtype1_last_1'].iloc[i]
    if np.isnan(histord_time_ordtype1_last_1):
        histord_time_ordtype1_last_1_month_list[i] = np.nan
        histord_time_ordtype1_last_1_hour_list[i] = np.nan
        histord_time_ordtype1_last_1_yday_list[i] = np.nan
        histord_time_ordtype1_last_1_timespan2holi_list[i] = np.nan
    else:
        tm_year, tm_mon, tm_day, tm_hour, tm_min, tm_sec, tm_wday, tm_yday = getTimeFromTimestamp(int(histord_time_ordtype1_last_1))
        histord_time_ordtype1_last_1_month_list[i] = tm_mon
        histord_time_ordtype1_last_1_hour_list[i] = tm_hour
        histord_time_ordtype1_last_1_yday_list[i] = tm_yday
        histord_time_ordtype1_last_1_timespan2holi_list[i] = getTimespanWithHolidays(tm_yday, yday_holi_list)
df['histord_time_ordtype1_last_1_month'] = histord_time_ordtype1_last_1_month_list
df['histord_time_ordtype1_last_1_hour'] = histord_time_ordtype1_last_1_hour_list
df['histord_time_ordtype1_last_1_yday'] = histord_time_ordtype1_last_1_yday_list
df['histord_time_ordtype1_last_1_timespan2holi'] = histord_time_ordtype1_last_1_timespan2holi_list

df['timespan2last_action1_indicator1'] = df['timespan2last_action1_mean'] * df['timespan2last_action1_std']
df['timespan2last_action5_indicator1'] = df['timespan2last_action5_mean'] * df['timespan2last_action5_std']
df['timespan2last_action6_indicator1'] = df['timespan2last_action6_mean'] * df['timespan2last_action6_std']
df['timespan2last_action7_indicator1'] = df['timespan2last_action7_mean'] * df['timespan2last_action7_std']
df['timespan2last_action8_indicator1'] = df['timespan2last_action8_mean'] * df['timespan2last_action8_std']
df['timespan2last_action9_indicator1'] = df['timespan2last_action9_mean'] * df['timespan2last_action9_std']
df['timespan2last_action1_indicator2'] = df['timespan2last_action1_len'] * df['timespan2last_action1_mean'] * df['timespan2last_action1_std']
df['timespan2last_action5_indicator2'] = df['timespan2last_action5_len'] * df['timespan2last_action5_mean'] * df['timespan2last_action5_std']
df['timespan2last_action6_indicator2'] = df['timespan2last_action6_len'] * df['timespan2last_action6_mean'] * df['timespan2last_action6_std']
df['timespan2last_action7_indicator2'] = df['timespan2last_action7_len'] * df['timespan2last_action7_mean'] * df['timespan2last_action7_std']
df['timespan2last_action8_indicator2'] = df['timespan2last_action8_len'] * df['timespan2last_action8_mean'] * df['timespan2last_action8_std']
df['timespan2last_action9_indicator2'] = df['timespan2last_action9_len'] * df['timespan2last_action9_mean'] * df['timespan2last_action9_std']

In [9]:
df.head(12)

,userid,histord_sum_all,histord_sum_0,histord_sum_1,histord_ratio1_0,histord_time_last_1,histord_time_last_1_year,histord_time_last_1_month,histord_time_last_1_hour,histord_time_last_1_yday,...,timespan2last_action6_indicator1,timespan2last_action7_indicator1,timespan2last_action8_indicator1,timespan2last_action9_indicator1,timespan2last_action1_indicator2,timespan2last_action5_indicator2,timespan2last_action6_indicator2,timespan2last_action7_indicator2,timespan2last_action8_indicator2,timespan2last_action9_indicator2
0,100000000371,1.0,1.0,0.0,0.500000,1.503444e+09,2017.0,8.0,7.0,235.0,...,0.000000e+00,0.000000,NaN,NaN,NaN,2.046600e+04,0.000000e+00,0.000000,NaN,NaN
1,100000001215,0.0,0.0,0.0,1.000000,NaN,NaN,NaN,NaN,NaN,...,0.000000e+00,NaN,NaN,NaN,4.047811e+03,0.000000e+00,0.000000e+00,NaN,NaN,NaN
2,100000001445,5.0,5.0,0.0,0.166667,1.491296e+09,2017.0,4.0,16.0,94.0,...,0.000000e+00,NaN,1.290791e+11,1.055948e+11,1.880284e+04,0.000000e+00,0.000000e+00,NaN,7.744747e+12,7.286038e+12
3,100000001575,3.0,2.0,1.0,0.666667,1.489587e+09,2017.0,3.0,22.0,74.0,...,0.000000e+00,NaN,8.777525e+09,NaN,8.953822e+06,0.000000e+00,0.000000e+00,NaN,2.369932e+11,NaN
4,100000001601,0.0,0.0,0.0,1.000000,NaN,NaN,NaN,NaN,NaN,...,6.766541e+12,NaN,1.921808e+11,2.300771e+11,0.000000e+00,NaN,8.119850e+13,NaN,1.345265e+12,1.380463e+12
5,100000001737,3.0,2.0,1.0,0.666667,1.494784e+09,2017.0,5.0,1.0,135.0,...,0.000000e+00,0.000000,6.641702e+08,7.132611e+08,1.074107e+04,5.800000e+03,0.000000e+00,0.000000,1.394757e+10,1.426522e+10
6,100000001829,1.0,1.0,0.0,0.500000,1.481480e+09,2016.0,12.0,2.0,347.0,...,0.000000e+00,NaN,1.226822e+10,NaN,5.916350e+04,0.000000e+00,0.000000e+00,NaN,1.472186e+11,NaN
7,100000002055,0.0,0.0,0.0,1.000000,NaN,NaN,NaN,NaN,NaN,...,8.117682e+08,NaN,6.234048e+08,8.017057e+08,0.000000e+00,2.451064e+09,8.929450e+09,NaN,2.493619e+09,2.405117e+09
8,100000002241,2.0,2.0,0.0,0.333333,1.482644e+09,2016.0,12.0,13.0,360.0,...,0.000000e+00,NaN,2.149821e+12,NaN,3.112146e+07,0.000000e+00,0.000000e+00,NaN,4.299642e+13,NaN
9,100000003067,0.0,0.0,0.0,1.000000,NaN,NaN,NaN,NaN,NaN,...,1.747059e+12,NaN,7.523860e+11,7.840685e+11,1.105000e+02,0.000000e+00,3.319411e+13,NaN,2.708590e+13,2.744240e+13


In [10]:
# write out data
#pd.DataFrame(df).to_csv('./result/data_train_1.csv', header=True, index=False)
pd.DataFrame(df).to_csv('./result/data_test_1.csv', header=True, index=False)